# Part 1 - initial data analysis and visualisation

## Import libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src import *

## Loading data

In [2]:
dataloader = CSVDataLoader('data/MSFT_2006-01-01_to_2018-01-01.csv')
raw_df = dataloader.load()

print(raw_df.info())
print()
print(raw_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3019 entries, 0 to 3018
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3019 non-null   object 
 1   Open    3019 non-null   float64
 2   High    3019 non-null   float64
 3   Low     3019 non-null   float64
 4   Close   3019 non-null   float64
 5   Volume  3019 non-null   int64  
 6   Name    3019 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 165.2+ KB
None

         Date   Open   High    Low  Close     Volume  Name
0  2006-01-03  26.25  27.00  26.10  26.84   79974418  MSFT
1  2006-01-04  26.77  27.08  26.77  26.97   57975661  MSFT
2  2006-01-05  26.96  27.13  26.91  26.99   48247610  MSFT
3  2006-01-06  26.89  27.00  26.49  26.91  100969092  MSFT
4  2006-01-09  26.93  27.07  26.76  26.86   55627836  MSFT


## Visualise time-series

In [6]:
StockVisualizer(title="Detail view").visualize(raw_df[:50])

# Compress dataframe (from each window of length 'w' select the day with the largest range Low - High)
rows, w = [], 100
for i in range(0, len(raw_df), w):
    window = raw_df.iloc[i:i+w]
    if window.empty:
        continue
    idx = (window["High"] - window["Low"]).idxmax()
    rows.append(raw_df.loc[idx])
compressed_df = pd.DataFrame(rows).reset_index(drop=True)

StockVisualizer(title="Global view").visualize(compressed_df)